# Description of Topics

In [1]:
#import the libraries
from constants import folder_meicogsci, folder_meicogsci_2019, folder_models, all_topics_names
import collections
import gensim
import pandas
import nltk
import os

In [2]:
#get the list of all files
all_files = os.listdir(folder_meicogsci)

In [3]:
#import model
model_21 = gensim.models.LdaModel.load(os.path.join(folder_models, "LDA_21"))
model_21.minimum_probability = 0.0

In [4]:
#calculate the most important words for each topic
current_model = model_21
words_represented = dict()
for topic_words in current_model.print_topics(num_topics=-1, num_words=40):
    current_topic = topic_words[0]
    words_represented[current_topic] = collections.defaultdict(list)
    all_words = topic_words[1].split("+")
    all_words = [w.split("*") for w in all_words]
    all_words = [[float(weight), word.replace('"', "").strip()] for weight, word in all_words]
    all_words_relative = []
    for weight, word in all_words:
        all_groups = current_model.get_term_topics(word)
        current_sum = sum([weight for topic, weight in all_groups])
        current_relative_weight = [weight/current_sum for topic, weight in all_groups if topic == current_topic][0]
        all_words_relative.append([current_relative_weight, weight, current_relative_weight*weight, word])
    words_represented[current_topic] = all_words_relative

In [5]:
# helping function for preprocessing text
lem = nltk.stem.WordNetLemmatizer()

In [6]:
# get the list of all titles with their topics
all_files_by_topic = []
for filename in all_files:
    with open(os.path.join(folder_meicogsci, filename)) as f:
        data = f.readlines()
        title = data[0].replace("... title: ", "").replace("\n", " ")
        data = " ".join(data).replace("... title: ", "").replace("\n", " ")
    data = data.lower()
    data_tok = nltk.tokenize.word_tokenize(data)
    data_tok = [lem.lemmatize(w) for w in data_tok]
    doc_topics, _, _ = model_21.get_document_topics(model_21.id2word.doc2bow(data_tok), per_word_topics=True)
    doc_topics = dict(doc_topics)
    new_row = []
    for i in range(21):
        if i in doc_topics:
            new_row.append(doc_topics[i])
        else:
            new_row.append(0)
    new_row.append(filename)
    new_row.append(title)
    all_files_by_topic.append(new_row)

In [7]:
all_files_by_topic = pandas.DataFrame(all_files_by_topic)

In [8]:
#create the helping function for priting the info about topic
def get_current_description_of_topic(model, i, words_represented, all_files_by_topic):
    print("Topic: " + all_topics_names[i].upper())
    scores = pandas.DataFrame(words_represented[i], columns=["Relative", "Absolute", "Score", "Word"])
    print("Score: " + ", ".join(scores.sort_values(by="Score", ascending=False).head(10)["Word"])[:-2])
    print("Absolute measure: " + ", ".join(scores.sort_values(by="Absolute", ascending=False).head(10)["Word"])[:-2])
    print("Relative measure: " + ", ".join(scores.sort_values(by="Relative", ascending=False).head(10)["Word"])[:-2])
    all_titles = [list(row) for row_id, row in 
                  all_files_by_topic.sort_values(by=i, ascending=False).head(5)[[i, 22]].iterrows()]
    print("Articles:")
    for value, title in all_titles:
        print("    * " + title + " (" + str(value*100)[:5] + ")")

In the following part, the topics are described. There are five parts in their description:
    
* Topic - the name of the topic
* Score - metrics from Absolute and Relative
* Absolute - importance of word inside the topic
* Relative - importance of word compated to this topic and other topics
* Articles - five articles, that have the highest value for this topic

In [9]:
get_current_description_of_topic(current_model, 0, words_represented, all_files_by_topic)

Topic: PITCH
Score: pitch, tone, musical, protein, stroke, native, grammar, curiosity, musician, spee
Absolute measure: language, brain, speech, tone, ability, human, native, musical, pitch, percepti
Relative measure: pitch, protein, musician, grammar, vocal, curiosity, musical, stroke, tone, syndro
Articles:
    * The effect of the harmonic context on the perception of pitch class  (87.86)
    * Prosody of Slovak Motherese  (83.38)
    * Talent for Accent: Is There a Correlation Between Phonetic Native and Foreign Language Aptitude?  (79.88)
    * A Critical Replica of Diana Deutsch's auditory Octave Illusion  (79.35)
    * Prosodic and Musical Abilities in English Language Pronunciation Learning  (75.33)


In [10]:
get_current_description_of_topic(current_model, 1, words_represented, all_files_by_topic)

Topic: MOVEMENT
Score: movement, eye, mu rhythm, bci, eye tracking, rhythm, body, painting, food, eye moveme
Absolute measure: movement, eye, system, motor, data, body, condition, rhythm, eeg, pictu
Relative measure: mu rhythm, bci, eye tracking, painting, food, eye movement, movement, rhythm, style, e
Articles:
    * Motor resonance based on desynchronization of the EEG mu rhythm  (90.42)
    * EEG mu rhythm desynchronisation: an electrophysiological evidence for mirror neurons activity  (80.02)
    * Eyetracking on paintings  (80.01)
    * Eye tracking on art paintings - differences between abstract and realistic paintings  (79.99)
    * Evaluation of consumer EEG device Emotiv EPOC  (79.47)


In [11]:
get_current_description_of_topic(current_model, 2, words_represented, all_files_by_topic)

Topic: CATEGORIZATION
Score: aesthetic, verbal, artwork, pigeon, app, art, array, pleasure, discrimination, ic
Absolute measure: aesthetic, verbal, experiment, art, artwork, subject, pigeon, part, app, peop
Relative measure: artwork, pigeon, icon, app, phonological loop, pleasure, liking, aesthetic, array, fluen
Articles:
    * Role of language in relational category discrimination  (89.41)
    * LANGUAGE AND PERCEPTION: The impact of conges-tion language modality for differentiating relational categories  (87.00)
    * Is Beauty Objective? An Experimental Study of Aesthetic Distance  (65.12)
    * The relation of elaboration and liking in aesthetic perception: Aesthetics as the pleasure of enacting meaning  (63.17)
    * A computational morphology approach to Croatian noun inflection with focus on gender prediction  (61.86)


In [12]:
get_current_description_of_topic(current_model, 3, words_represented, all_files_by_topic)

Topic: MODELING
Score: model, simulation, strategy, need, system, theory, communication, author, agent, interacti
Absolute measure: model, system, theory, cognitive, human, need, simulation, strategy, agent, informati
Relative measure: simulation, strategy, model, need, author, computational, communication, framework, step, behavio
Articles:
    * Towards opening black boxes in behavioral science through simulation  (83.89)
    * Can Economic Models Be Epistemic Vehicles?  (80.93)
    * Complex Skill Development with Starcraft 2  (80.65)
    * Cognitive aspects of StarCraft 2  (79.16)
    * Cognitive aspects of Starcraft 2  (72.14)


In [13]:
get_current_description_of_topic(current_model, 4, words_represented, all_files_by_topic)

Topic: SOCIETY
Score: social, behavior, psychotherapy, human, interaction, specie, field, theoretical, individual, rewa
Absolute measure: social, human, behavior, cognitive, interaction, theory, field, individual, model, analys
Relative measure: psychotherapy, social, specie, behavior, awareness, theoretical, interaction, literature, architecture, rewa
Articles:
    * !!The Costs of Being Behind a Screen – Could Social Media Use be Addictive?  (88.51)
    * Darwin, The Functionalist Approach to Modern Psychology and the Central European Heritage  (87.69)
    * Agent-based Modeling: A Novel Practitioner’s Generative View on Social Systems  (84.15)
    * Modeling of social learning in ravens  (83.02)
    * Simulating the Emergence of Social Complexity Using Agent-Based Modelling  (82.18)


In [14]:
get_current_description_of_topic(current_model, 5, words_represented, all_files_by_topic)

Topic: DECISION MAKING
Score: decision, decision making, making, moral, user, risk, satisfaction, bias, emotional, situati
Absolute measure: decision, decision making, user, making, participant, moral, emotional, group, people, situati
Relative measure: moral, decision making, making, satisfaction, decision, bias, risk, story, user, affecti
Articles:
    * Illusion of Control and Persistence  (83.95)
    * Empathy and Prosocial Behavior across the Lifespan  (80.53)
    * The Gamification of a Tax Experiment  (80.49)
    * Life Satisfaction in Old Age: Comparing the Successful and Harmonious Aging Approaches  (79.90)
    * Organizational Failure and Decision Making  (79.55)


In [15]:
get_current_description_of_topic(current_model, 6, words_represented, all_files_by_topic)

Topic: NON-TYPICAL
Score: patient, disease, response, dancer, asd, mental imagery, hand, imagery, blood, hea
Absolute measure: patient, response, participant, disease, condition, group, hand, stimulus, problem, activi
Relative measure: asd, mental imagery, heart, disease, dancer, blood, diagnosis, skin, imagery, patie
Articles:
    * Differences in EEG and Autonomic Nervous System Activity as a Potential Biomarker in Depressive Disorders  (86.92)
    * Diagnosing of Narcolepsy in Slovenia  (85.03)
    * Movement mental imagery to groovy and non-groovy music in dancers and non-dancers  (83.84)
    * Discovering Biomarkers of Alzheimer Disease  (81.82)
    * Action observation and motor imagery effects on motor learning  (71.51)


In [16]:
get_current_description_of_topic(current_model, 7, words_represented, all_files_by_topic)

Topic: NEUROSCIENCE
Score: receptor, brain, eeg, subunit, region, fmri, connectivity, amplitude, functional, functi
Absolute measure: brain, task, cognitive, eeg, method, data, memory, stimulus, receptor, functi
Relative measure: subunit, receptor, connectivity, amplitude, region, fmri, eeg, functional, working memory, rhyt
Articles:
    * Duration of tissue fixation does not influence immunohistochemical staining of GABA-A receptor subunits in human hippocampus  (89.04)
    * Gabaa Receptor Subunit Distribution & Variants  (87.67)
    * Investigating brain functional connectivity with simultaneous EEG and fMRI recordings  (86.47)
    * GABA-A Receptors in Anxiety and Fear  (86.18)
    * EEG-fMRI source localization  (79.96)


In [17]:
get_current_description_of_topic(current_model, 8, words_represented, all_files_by_topic)

Topic: CONSTRUCTIVISM
Score: experience, emotion, consciousness, phenomenological, conscious, interview, pain, knowledge, need, mi
Absolute measure: experience, emotion, process, knowledge, theory, method, consciousness, phenomenological, question, conscio
Relative measure: conscious, phenomenological, consciousness, pain, experience, interview, emotion, thought, view, explanati
Articles:
    * Music Emotions as Conceptual Acts?  (90.64)
    * In Search for a Middle Way  (89.62)
    * Experience of Sexual Fantasy  (88.60)
    * Pain Asymbolia and Nature of Pain  (85.92)
    * Rethinking materialism: an outline of integrative science  (82.77)


In [18]:
get_current_description_of_topic(current_model, 9, words_represented, all_files_by_topic)

Topic: HEALTH
Score: sleep, memory, deprivation, sleep deprivation, false, placebo, hour, innovation, placebo effect, d
Absolute measure: memory, sleep, effect, deprivation, sleep deprivation, time, false, placebo, cognitive, experime
Relative measure: deprivation, sleep deprivation, placebo effect, false, placebo, sleep, hour, cognitive function, innovation, bli
Articles:
    * Effects of Sleep Deprivation on Motor and Cognitive Performance  (71.43)
    * Placebo Effect Triggers and Their Possible Merit in Effectiveness of Complementary and Alternative Medicine Practices  (70.48)
    * Effects of Humor and Laughter Group Therapy on People with Depressive Disorder  (66.41)
    * The influence of sleep deficit on false memory phenomenon  (65.10)
    * Does Daytime Saving Time Steal Your Time?  (60.70)


In [19]:
get_current_description_of_topic(current_model, 10, words_represented, all_files_by_topic)

Topic: PERCEPTION
Score: target, color, visual, death, attention, stimulus, screen, distractors, attentional, sce
Absolute measure: visual, stimulus, attention, target, task, participant, color, subject, time, obje
Relative measure: death, distractors, mortality, color, scene, target, attentional, screen, reaction time, nov
Articles:
    * ===============      !!!!PRIMING OF VISUAL ATTENTION IN DYNAMIC VISUAL SCENES- AN EXPERIMENTAL STUDY USING EYE TRACKING  (84.62)
    * The Effect of Haptic Guidance on Motor Learning of Rhythmic Sequences  (73.79)
    * The Meaning of Dying  (73.68)
    * !!!! A Visuo-Spatial Cueing Experiment for Unconscious Peripheral Vision  (72.70)
    * Visual attention and action 3D video games  (72.64)


In [20]:
get_current_description_of_topic(current_model, 11, words_represented, all_files_by_topic)

Topic: LEARNING
Score: child, cognitive, student, cognitive control, educational, control, school, education, group, ta
Absolute measure: cognitive, task, group, control, child, student, performance, process, cognitive control, educati
Relative measure: educational, cognitive control, school, education, child, management, student, stress, impairment, contr
Articles:
    * 3D TRIANGULATION AS A METHOD FOR EVALUATION OF DYSTONIA IN PATIENTS WITH TORTICOLLIS  (85.77)
    * Treating children with epilepsy  (85.76)
    * Influence of alternative education on cooperativeness  (79.12)
    * Effects of cognitive and motor tasks on gait parameters and stride variability during treadmill walking  (78.95)
    * The Embodied Cognition Explored on a Case Study  (69.40)


In [21]:
get_current_description_of_topic(current_model, 12, words_represented, all_files_by_topic)

Topic: VERBALIZATION
Score: sound, meditation, metaphor, prediction, schizophrenia, beat, sentence, mindfulness meditation, creative, prediction err
Absolute measure: sound, meditation, prediction, sentence, metaphor, schizophrenia, beat, model, sense, se
Relative measure: prediction error, mindfulness meditation, meditation, aha, metaphor, meditators, beat, sound, schizophrenia, predicti
Articles:
    * The Aesthetic Reset – Mechanism and Function of the Aesthetic Experience in the Predictive Mind  (82.35)
    * 'Fictive' sounds in language: processing of abstract sentences encoding fictive sounds evoke sound representations  (80.97)
    * Speech alterations in schizophrenia  (57.00)
    * Towards a Sensorimotor Account of Schizophrenic Self-disorders  (54.56)
    * Challenges for Embodiment: the Case of Language Change  (54.27)


In [22]:
get_current_description_of_topic(current_model, 13, words_represented, all_files_by_topic)

Topic: LANGUAGE
Score: word, language, text, comprehension, lexical, sentence, semantic, slovene, meaning, linguist
Absolute measure: language, word, sentence, text, processing, information, linguistic, semantic, comprehension, participa
Relative measure: syntactic, lexical, slovene, word, comprehension, text, corpus, verb, semantic, langua
Articles:
    * !!!!Acquisition of Slovene Adjective Inflection and Semantics  (84.24)
    * Using Clustering to Determine the Basic Structure of Natural Language  (81.76)
    * Slovene compounds: Towards a psycholinguistic approach  (81.43)
    * Meaning in Morphological Decomposition  (80.90)
    * Acquisition Of Slovene Adjective Inflection And Semantics  (78.51)


In [23]:
get_current_description_of_topic(current_model, 14, words_represented, all_files_by_topic)

Topic: REASONING
Score: spatial, working memory, memory, information, capacity, object, probability, scheme, change, scien
Absolute measure: memory, information, cognitive, object, change, theory, spatial, science, visual, working memo
Relative measure: scheme, probability, spatial, agency, capacity, creation, working memory, scientist, problem solving, fr
Articles:
    * Memory: A Paradigm for Reductionism?  (77.11)
    * The Medium Is the Message: How Presentation Formats Help People Ask Better Questions  (75.39)
    * Some “Internal” Problems of Normative Theories of Thinking and Reasoning  (71.27)
    * Making Sense  (70.73)
    * Learning Environmental Probabilities in Changing Worlds  (69.61)


In [24]:
get_current_description_of_topic(current_model, 15, words_represented, all_files_by_topic)

Topic: ATTENTION
Score: face, attention, gaze, dog, female, image, male, eye, mirror neuron, mirr
Absolute measure: attention, task, eye, image, face, participant, emotional, gaze, stimulus, recogniti
Relative measure: face, dog, female, mirror neuron, gaze, male, mirror, conflict, fixation, attenti
Articles:
    * EYE MOVEMENT CORRELATES FOR THE FACE RECOGNITION  (92.04)
    * Enhancing the Concealed Information Test via Emphasis on Semantic Associations  (66.81)
    * Towards an Understanding of the Emotional Modulation Effects of Focused Attention Meditation  (61.57)
    * Romantic Love and Visual Recognition  (53.44)
    * Emergence of Visual Consciousness in ADHD Children  (52.67)


In [25]:
get_current_description_of_topic(current_model, 16, words_represented, all_files_by_topic)

Topic: SYSTEM
Score: legal, gesture, article, robot, rule, dance, algorithm, vowel, system, machi
Absolute measure: system, human, legal, robot, algorithm, gesture, rule, agent, language, wo
Relative measure: legal, vowel, gesture, article, dance, reasoning, rule, code, machine, devi
Articles:
    * Adaptive Gesture Recognition System, transforming Dance Performance into Music  (82.87)
    * Logical cognitive analysis of legal reasoning  (80.40)
    * Prosodic Cues in a Reference Resolution Algorithm  (66.88)
    * Communication platform for agents in heterogeneous environment  (62.99)
    * Explaining Robot Actions  (61.54)


In [26]:
get_current_description_of_topic(current_model, 17, words_represented, all_files_by_topic)

Topic: TASKS
Score: game, player, serious, empathy, design, message, trust, norm, physic, listen
Absolute measure: game, player, design, participant, empathy, level, serious, message, trust, peop
Relative measure: serious, physic, message, game, trust, norm, listener, empathy, player, desi
Articles:
    * Playing and Learning  (87.72)
    * Fetal Testosterone and Autism  (78.53)
    * Empathy of Catholics  (74.58)
    * Explaining the Divergent Effects of Socioeconomic Status on Prosocial and Antisocial Behaviour in Economic Decision Making  (70.69)
    * “Trust development in repeated interactions: The relationship between implicit and explicit measures of trust and personality variables.“  (68.01)


In [27]:
get_current_description_of_topic(current_model, 18, words_represented, all_files_by_topic)

Topic: TMS
Score: tm, stimulation, cortex, motor, rtms, magnetic, cortical, motor cortex, brain, protoc
Absolute measure: brain, motor, tm, stimulation, cortex, area, effect, cortical, subject, depressi
Relative measure: rtms, transcranial, transcranial magnetic stimulation, tm, excitability, stimulation, motor cortex, magnetic, prefrontal cortex, pul
Articles:
    * Plasticity of the primary motor cortex in learning new movement trajectories  (86.46)
    * Making a Case for Theta Burst Stimulation in Treatment of Depression  (86.01)
    * Combining Electroencephalography With Transcranial Magnetic Stimulation As a New Step in Understanding Mechanisms of Non-invasive Brain Stimulation Techniques  (77.39)
    * Effects of TMS Protocols on the Neuroplasticity of the Primary Motor Cortex  (77.26)
    * Measuring Direct Effect of cTBS over Right DLPFC on Processing Cognitive-emotional Tasks and Its Efficiency as a Depression Treatment over Time  (76.84)


In [28]:
get_current_description_of_topic(current_model, 19, words_represented, all_files_by_topic)

Topic: NEURAL NETWORKS
Score: network, problem, neuron, layer, cell, neural network, neural, matrix, representation, creativi
Absolute measure: network, problem, neuron, neural, representation, neural network, model, cell, layer, da
Relative measure: layer, dataset, matrix, creativity, cell, neuron, problem solving, mathematical, network, neural netwo
Articles:
    * Toward Evolutionary Design Optimization by Cellular Representations with Implicit Ontogeny  (89.33)
    * The Role of Mental Representations in Problem Solving Process  (76.51)
    * USING DEEP BELIEF NETWORK IN OBJECT RECOGNITION  (73.63)
    * Testing the hierarchical neural network DBN in invariant object recognition  (69.88)
    * Blockchain-based IoT Data Storage  (65.83)


In [29]:
get_current_description_of_topic(current_model, 20, words_represented, all_files_by_topic)

Topic: REINFORCMENT LEARNING
Score: agent, action, reward, robot, reinforcement, environment, reinforcement learning, learning, motivation, act
Absolute measure: agent, action, robot, environment, learning, reward, neural, image, model, sta
Relative measure: reinforcement learning, reinforcement, actor, reward, agent, motivation, mobile, robot, action, fr
Articles:
    * Modeling of behavior of cognitive agents in a virtual world in the TerraSim simulation environment  (92.10)
    * Inter-agent communication as a mean of achieving objectives  (87.43)
    * Goal-driven developmental learning on a mobile robot  (85.42)
    * Deep Reinforcement Learning for Computer Games  (83.65)
    * Learning the Instructions How to Reach with a Simulated Robotic Arm  (81.81)
